# Fundamentos de RDD (Resilient Distributed Datasets)

## Objetivos de Aprendizaje
- Entender qué es un RDD y por qué es importante
- Crear RDDs desde diferentes fuentes
- Aplicar transformaciones y acciones básicas
- Comprender el concepto de particiones y distribución

## Prerequisitos
- `00_setup/01_environment_check.ipynb`
- `00_setup/02_spark_basics.ipynb`

## Tiempo Estimado
⏱️ 45 minutos

## Módulo AWS Academy Relacionado
📚 Módulo 9: Big Data Processing - Fundamentos de procesamiento distribuido

---
# === SECCIÓN 1 ===
## 1. ¿Qué es un RDD?

### Explicación Conceptual

**RDD (Resilient Distributed Dataset)** es la abstracción fundamental de Spark. Es una colección de elementos:
- **Resilient**: Si una parte falla, se puede reconstruir
- **Distributed**: Los datos están repartidos en múltiples máquinas
- **Dataset**: Es una colección de datos

**Analogía del mundo real:** Imagina que tienes que contar todas las palabras en una biblioteca de 1 millón de libros. Un RDD te permite dividir esos libros entre 100 personas (workers), cada una cuenta sus libros (particiones), y al final sumas los resultados. Si alguien pierde su cuenta, puede volver a hacerla porque sabe qué libros tenía.

### ¿Por qué aprender RDDs si existen DataFrames?
- DataFrames usan RDDs internamente
- Entender RDDs ayuda a debuggear problemas
- Algunas operaciones de bajo nivel requieren RDDs
- Código legacy usa RDDs

In [ ]:
# Importamos SparkSession y SparkContext
from pyspark.sql import SparkSession

# Creamos la sesion de Spark
spark = SparkSession.builder \
    .appName("RDD_Fundamentals") \
    .master("local[*]") \
    .getOrCreate()

# SparkContext es el punto de entrada para crear RDDs
# sc es una convencion comun para nombrar el SparkContext
sc = spark.sparkContext

# Reducimos logs para ver mejor los outputs
sc.setLogLevel("WARN")

print("SparkContext creado")
print(f"Application ID: {sc.applicationId}")
print(f"Master: {sc.master}")

---
# === SECCIÓN 2 ===
## 2. Crear RDDs

### Explicación Conceptual
Hay dos formas principales de crear un RDD:
1. **parallelize()**: Desde una colección de Python (listas, tuplas)
2. **textFile()**: Desde archivos externos (CSV, TXT, etc.)

**Analogía:** Es como crear una lista de tareas. Puedes escribirla tú mismo (parallelize) o importarla de un archivo (textFile).

In [ ]:
# METODO 1: Crear RDD desde una lista de Python
# parallelize() distribuye los datos en el cluster

# Lista de numeros del 1 al 10
numeros = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Creamos el RDD con 4 particiones
# Las particiones son "pedazos" del RDD distribuidos en workers
rdd_numeros = sc.parallelize(numeros, 4)

# Verificamos que se creo correctamente
print(f"Tipo: {type(rdd_numeros)}")
print(f"Numero de particiones: {rdd_numeros.getNumPartitions()}")

# collect() es una ACCION que trae todos los datos al driver
# CUIDADO: No usar collect() con datos grandes!
print(f"Contenido: {rdd_numeros.collect()}")

# Output esperado:
# Tipo: <class 'pyspark.rdd.RDD'>
# Numero de particiones: 4
# Contenido: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
# Ver como estan distribuidos los datos en cada particion
# glom() agrupa elementos por particion

particiones = rdd_numeros.glom().collect()

print("Distribucion de datos por particion:")
for i, particion in enumerate(particiones):
    print(f"  Particion {i}: {particion}")

# Output esperado:
# Particion 0: [1, 2]
# Particion 1: [3, 4, 5]
# Particion 2: [6, 7]
# Particion 3: [8, 9, 10]

In [ ]:
# METODO 2: Crear RDD desde texto
# Primero creamos un archivo de ejemplo

# Datos de ejemplo: ventas por linea
texto_ventas = """2024-01-01,Laptop,15000
2024-01-01,Mouse,350
2024-01-02,Monitor,8000
2024-01-02,Teclado,800
2024-01-03,Laptop,15000"""

# Guardamos el archivo
ruta_archivo = "/home/jovyan/data/sample/ventas_rdd.txt"
with open(ruta_archivo, 'w') as f:
    f.write(texto_ventas)

print(f"Archivo creado en: {ruta_archivo}")

In [ ]:
# Leemos el archivo como RDD
# textFile() lee cada linea como un elemento del RDD
rdd_ventas = sc.textFile(ruta_archivo)

# Mostramos las primeras lineas
# take(n) trae solo n elementos (mas seguro que collect)
print("Primeras 3 lineas del archivo:")
for linea in rdd_ventas.take(3):
    print(f"  {linea}")

print(f"\nTotal de lineas: {rdd_ventas.count()}")

# Output esperado:
# Primeras 3 lineas del archivo:
#   2024-01-01,Laptop,15000
#   2024-01-01,Mouse,350
#   2024-01-02,Monitor,8000

---
# === SECCIÓN 3 ===
## 3. Transformaciones Básicas

### Explicación Conceptual
Las transformaciones crean un NUEVO RDD a partir de uno existente. Son **lazy** (no se ejecutan hasta que hay una acción).

**Transformaciones comunes:**
- `map()`: Aplica una función a cada elemento
- `filter()`: Filtra elementos según una condición
- `flatMap()`: Como map pero "aplana" los resultados

**Analogía:** Las transformaciones son como instrucciones en una receta. No cocinas nada hasta que decides servir el plato (acción).

In [ ]:
# MAP: Aplica una funcion a cada elemento
# Ejemplo: Elevar cada numero al cuadrado

# La funcion lambda toma un elemento y retorna su cuadrado
rdd_cuadrados = rdd_numeros.map(lambda x: x ** 2)

print("Numeros originales:", rdd_numeros.collect())
print("Numeros al cuadrado:", rdd_cuadrados.collect())

# Output esperado:
# Numeros originales: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# Numeros al cuadrado: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [ ]:
# MAP con funcion mas compleja
# Parsear las lineas de ventas

def parsear_venta(linea):
    """Convierte una linea de texto en una tupla estructurada"""
    # split() divide el string por comas
    partes = linea.split(",")
    # Retornamos una tupla (fecha, producto, precio)
    return (partes[0], partes[1], int(partes[2]))

# Aplicamos la funcion a cada linea
rdd_ventas_parsed = rdd_ventas.map(parsear_venta)

print("Ventas parseadas:")
for venta in rdd_ventas_parsed.collect():
    print(f"  Fecha: {venta[0]}, Producto: {venta[1]}, Precio: ${venta[2]:,}")

# Output esperado:
# Fecha: 2024-01-01, Producto: Laptop, Precio: $15,000

In [ ]:
# FILTER: Selecciona elementos que cumplen una condicion

# Filtrar solo numeros pares
# lambda x: x % 2 == 0 retorna True si x es par
rdd_pares = rdd_numeros.filter(lambda x: x % 2 == 0)

print("Numeros pares:", rdd_pares.collect())

# Filtrar ventas mayores a 1000
rdd_ventas_grandes = rdd_ventas_parsed.filter(lambda v: v[2] > 1000)

print("\nVentas mayores a $1,000:")
for venta in rdd_ventas_grandes.collect():
    print(f"  {venta[1]}: ${venta[2]:,}")

# Output esperado:
# Numeros pares: [2, 4, 6, 8, 10]
# Ventas mayores a $1,000:
#   Laptop: $15,000
#   Monitor: $8,000
#   Laptop: $15,000

In [ ]:
# FLATMAP: Map que "aplana" los resultados
# Util cuando cada elemento genera multiples resultados

# Ejemplo: Dividir oraciones en palabras
oraciones = ["Hola mundo", "Apache Spark es genial", "Big Data"]
rdd_oraciones = sc.parallelize(oraciones)

# Con map: cada oracion se convierte en una lista de palabras
rdd_con_map = rdd_oraciones.map(lambda x: x.split())
print("Con map (listas anidadas):", rdd_con_map.collect())

# Con flatMap: las listas se "aplanan" en elementos individuales
rdd_con_flatmap = rdd_oraciones.flatMap(lambda x: x.split())
print("Con flatMap (lista plana):", rdd_con_flatmap.collect())

# Output esperado:
# Con map: [['Hola', 'mundo'], ['Apache', 'Spark', 'es', 'genial'], ['Big', 'Data']]
# Con flatMap: ['Hola', 'mundo', 'Apache', 'Spark', 'es', 'genial', 'Big', 'Data']

---
# === SECCIÓN 4 ===
## 4. Acciones Comunes

### Explicación Conceptual
Las acciones **disparan la ejecución** de todas las transformaciones pendientes y **retornan un resultado** al driver.

**Acciones comunes:**
- `collect()`: Trae todos los datos al driver (⚠️ cuidado con datos grandes)
- `count()`: Cuenta elementos
- `take(n)`: Trae los primeros n elementos
- `reduce()`: Combina elementos usando una función
- `foreach()`: Ejecuta una función en cada elemento

In [ ]:
# COUNT: Cuenta el numero de elementos
total = rdd_numeros.count()
print(f"Total de numeros: {total}")

# FIRST: Obtiene el primer elemento
primero = rdd_numeros.first()
print(f"Primer elemento: {primero}")

# TAKE: Obtiene los primeros n elementos
primeros_5 = rdd_numeros.take(5)
print(f"Primeros 5 elementos: {primeros_5}")

# TOP: Obtiene los n elementos mas grandes
top_3 = rdd_numeros.top(3)
print(f"Top 3 elementos: {top_3}")

# Output esperado:
# Total de numeros: 10
# Primer elemento: 1
# Primeros 5 elementos: [1, 2, 3, 4, 5]
# Top 3 elementos: [10, 9, 8]

In [ ]:
# REDUCE: Combina todos los elementos usando una funcion
# La funcion toma 2 elementos y retorna 1

# Sumar todos los numeros
# lambda a, b: a + b toma dos numeros y retorna su suma
suma_total = rdd_numeros.reduce(lambda a, b: a + b)
print(f"Suma de todos los numeros: {suma_total}")

# Encontrar el maximo
maximo = rdd_numeros.reduce(lambda a, b: a if a > b else b)
print(f"Numero maximo: {maximo}")

# Concatenar strings
palabras = sc.parallelize(["Hola", "Mundo", "Spark"])
concatenado = palabras.reduce(lambda a, b: a + " " + b)
print(f"Palabras concatenadas: {concatenado}")

# Output esperado:
# Suma de todos los numeros: 55
# Numero maximo: 10
# Palabras concatenadas: Hola Mundo Spark

In [ ]:
# Otras acciones utiles

# sum(): Suma todos los elementos (solo para numericos)
print(f"Suma: {rdd_numeros.sum()}")

# mean(): Promedio
print(f"Promedio: {rdd_numeros.mean()}")

# max() y min(): Maximo y minimo
print(f"Maximo: {rdd_numeros.max()}")
print(f"Minimo: {rdd_numeros.min()}")

# stdev(): Desviacion estandar
print(f"Desviacion estandar: {rdd_numeros.stdev():.2f}")

# Output esperado:
# Suma: 55
# Promedio: 5.5
# Maximo: 10
# Minimo: 1
# Desviacion estandar: 2.87

---
# === SECCIÓN 5 ===
## 5. Operaciones de Pares Clave-Valor

### Explicación Conceptual
Muchas operaciones de Big Data trabajan con pares **(clave, valor)**. Spark tiene operaciones especiales para este tipo de datos.

**Analogía:** Es como organizar recibos por categoría. La categoría es la "clave" y el monto es el "valor". Luego puedes sumar todos los montos por categoría.

**Operaciones comunes:**
- `reduceByKey()`: Agrupa y reduce por clave
- `groupByKey()`: Agrupa valores por clave
- `sortByKey()`: Ordena por clave
- `countByKey()`: Cuenta elementos por clave

In [ ]:
# Crear un RDD de pares (producto, precio)
ventas_pares = [
    ("Laptop", 15000),
    ("Mouse", 350),
    ("Laptop", 15000),
    ("Mouse", 350),
    ("Monitor", 8000),
    ("Mouse", 350),
    ("Laptop", 14500)
]

rdd_ventas_pares = sc.parallelize(ventas_pares)

print("Ventas originales:")
for venta in rdd_ventas_pares.collect():
    print(f"  {venta}")

In [ ]:
# REDUCEBYKEY: Agrupa por clave y reduce los valores
# Es la operacion mas comun y eficiente para agregaciones

# Sumar ventas por producto
# Para cada clave (producto), suma los valores (precios)
ventas_por_producto = rdd_ventas_pares.reduceByKey(lambda a, b: a + b)

print("Ventas totales por producto:")
for producto, total in ventas_por_producto.collect():
    print(f"  {producto}: ${total:,}")

# Output esperado:
# Laptop: $44,500
# Mouse: $1,050
# Monitor: $8,000

In [ ]:
# COUNTBYKEY: Cuenta ocurrencias por clave
conteo = rdd_ventas_pares.countByKey()

print("Numero de ventas por producto:")
for producto, cantidad in conteo.items():
    print(f"  {producto}: {cantidad} ventas")

# Output esperado:
# Laptop: 3 ventas
# Mouse: 3 ventas
# Monitor: 1 ventas

In [ ]:
# GROUPBYKEY: Agrupa todos los valores por clave
# CUIDADO: Menos eficiente que reduceByKey para agregaciones

agrupado = rdd_ventas_pares.groupByKey()

print("Valores agrupados por producto:")
for producto, valores in agrupado.collect():
    # valores es un iterador, lo convertimos a lista
    lista_valores = list(valores)
    print(f"  {producto}: {lista_valores}")

# Output esperado:
# Laptop: [15000, 15000, 14500]
# Mouse: [350, 350, 350]
# Monitor: [8000]

In [ ]:
# SORTBYKEY: Ordena por clave
ordenado = rdd_ventas_pares.sortByKey()

print("Ventas ordenadas alfabeticamente por producto:")
for venta in ordenado.collect():
    print(f"  {venta}")

# Ordenar descendente
ordenado_desc = rdd_ventas_pares.sortByKey(ascending=False)
print("\nOrdenado Z-A:")
for venta in ordenado_desc.collect():
    print(f"  {venta}")

---
# === SECCIÓN 6 ===
## 6. Ejemplo Práctico: Word Count

### Explicación Conceptual
El "Word Count" es el "Hola Mundo" del Big Data. Consiste en contar cuántas veces aparece cada palabra en un texto. Es simple pero demuestra el patrón Map-Reduce usado en procesamiento distribuido.

In [ ]:
# Texto de ejemplo
texto = """
Apache Spark es un framework de procesamiento de datos
Spark puede procesar datos en memoria
Spark es muy rapido para Big Data
Big Data es el futuro del procesamiento de datos
"""

# Guardamos el texto en un archivo
ruta_texto = "/home/jovyan/data/sample/texto_wordcount.txt"
with open(ruta_texto, 'w') as f:
    f.write(texto)

# Leemos el archivo como RDD
rdd_texto = sc.textFile(ruta_texto)
print("Lineas del texto:")
for linea in rdd_texto.collect():
    if linea.strip():  # Solo lineas no vacias
        print(f"  {linea}")

In [ ]:
# WORD COUNT - Paso a paso

# Paso 1: Dividir cada linea en palabras (flatMap)
# flatMap aplana el resultado: [["a", "b"], ["c"]] -> ["a", "b", "c"]
palabras = rdd_texto.flatMap(lambda linea: linea.lower().split())
print(f"Paso 1 - Total de palabras: {palabras.count()}")
print(f"Muestra: {palabras.take(10)}")

# Paso 2: Crear pares (palabra, 1)
# Cada palabra se convierte en un par clave-valor
pares = palabras.map(lambda palabra: (palabra, 1))
print(f"\nPaso 2 - Pares creados:")
print(f"Muestra: {pares.take(5)}")

# Paso 3: Sumar por clave (reduceByKey)
# Agrupa las palabras iguales y suma sus conteos
conteos = pares.reduceByKey(lambda a, b: a + b)
print(f"\nPaso 3 - Palabras unicas: {conteos.count()}")

# Paso 4: Ordenar por conteo (descendente)
# Intercambiamos clave-valor para ordenar por conteo
ordenado = conteos.map(lambda x: (x[1], x[0])) \
                  .sortByKey(ascending=False) \
                  .map(lambda x: (x[1], x[0]))

print("\nTop 10 palabras mas frecuentes:")
for palabra, conteo in ordenado.take(10):
    print(f"  '{palabra}': {conteo}")

In [ ]:
# Version compacta del Word Count (una linea encadenada)

resultado = sc.textFile(ruta_texto) \
    .flatMap(lambda l: l.lower().split()) \
    .map(lambda w: (w, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: x[1], ascending=False)

print("Word Count (version compacta):")
for palabra, conteo in resultado.take(5):
    print(f"  {palabra}: {conteo}")

---
# === EJERCICIOS PRÁCTICOS ===

### 🎯 Ejercicio 6.1: Filtrar y Transformar

Dado el RDD de números del 1 al 20:
1. Filtra solo los números divisibles por 3
2. Multiplica cada número filtrado por 10
3. Calcula la suma total

**Pistas:**
- `x % 3 == 0` verifica si x es divisible por 3
- Encadena `filter()`, `map()`, y `reduce()` o `sum()`

In [ ]:
# TODO: Completa el ejercicio
rdd_1_20 = sc.parallelize(range(1, 21))

# Tu codigo aqui:


### ✅ Solución Ejercicio 6.1

In [ ]:
# Solucion: Filtrar y transformar

rdd_1_20 = sc.parallelize(range(1, 21))

# Paso 1: Filtrar divisibles por 3
divisibles_3 = rdd_1_20.filter(lambda x: x % 3 == 0)
print(f"Divisibles por 3: {divisibles_3.collect()}")

# Paso 2: Multiplicar por 10
multiplicados = divisibles_3.map(lambda x: x * 10)
print(f"Multiplicados por 10: {multiplicados.collect()}")

# Paso 3: Suma total
suma = multiplicados.sum()
print(f"Suma total: {suma}")

# Version encadenada:
resultado = sc.parallelize(range(1, 21)) \
    .filter(lambda x: x % 3 == 0) \
    .map(lambda x: x * 10) \
    .sum()

print(f"\nResultado (encadenado): {resultado}")

# Output esperado:
# Divisibles por 3: [3, 6, 9, 12, 15, 18]
# Multiplicados por 10: [30, 60, 90, 120, 150, 180]
# Suma total: 630

### 🎯 Ejercicio 6.2: Análisis de Ventas con RDD

Dado el siguiente RDD de ventas `(producto, cantidad, precio_unitario)`:
1. Calcula el ingreso total por producto (cantidad × precio)
2. Encuentra el producto con mayor ingreso

**Pistas:**
- Usa `map()` para calcular ingreso y crear pares (producto, ingreso)
- Usa `reduceByKey()` para sumar por producto
- Usa `max()` con una función key

In [ ]:
# TODO: Completa el ejercicio
ventas_data = [
    ("Laptop", 2, 15000),
    ("Mouse", 10, 350),
    ("Laptop", 1, 15000),
    ("Monitor", 3, 8000),
    ("Mouse", 5, 350)
]

rdd_ventas = sc.parallelize(ventas_data)

# Tu codigo aqui:


### ✅ Solución Ejercicio 6.2

In [ ]:
# Solucion: Analisis de ventas

ventas_data = [
    ("Laptop", 2, 15000),
    ("Mouse", 10, 350),
    ("Laptop", 1, 15000),
    ("Monitor", 3, 8000),
    ("Mouse", 5, 350)
]

rdd_ventas = sc.parallelize(ventas_data)

# Paso 1: Calcular ingreso por venta y crear par (producto, ingreso)
# venta[0] = producto, venta[1] = cantidad, venta[2] = precio
rdd_ingresos = rdd_ventas.map(lambda v: (v[0], v[1] * v[2]))
print("Ingresos por venta:")
for producto, ingreso in rdd_ingresos.collect():
    print(f"  {producto}: ${ingreso:,}")

# Paso 2: Sumar ingresos por producto
ingresos_por_producto = rdd_ingresos.reduceByKey(lambda a, b: a + b)
print("\nIngreso total por producto:")
for producto, total in ingresos_por_producto.collect():
    print(f"  {producto}: ${total:,}")

# Paso 3: Encontrar el producto con mayor ingreso
# max() con key=lambda x: x[1] compara por el segundo elemento (ingreso)
mejor_producto = ingresos_por_producto.max(key=lambda x: x[1])
print(f"\nProducto con mayor ingreso: {mejor_producto[0]} (${mejor_producto[1]:,})")

# Output esperado:
# Ingreso total por producto:
#   Laptop: $45,000
#   Mouse: $5,250
#   Monitor: $24,000
# Producto con mayor ingreso: Laptop ($45,000)

### 🎯 Ejercicio 6.3: Word Count Mejorado

Mejora el Word Count para:
1. Eliminar palabras de menos de 3 letras
2. Eliminar signos de puntuación
3. Mostrar solo palabras que aparecen más de 1 vez

**Pistas:**
- Usa `re` (expresiones regulares) o `str.isalpha()`
- `len(palabra) >= 3` filtra por longitud

In [ ]:
# TODO: Completa el ejercicio
texto_ejercicio = """
Python es un lenguaje de programacion muy popular.
Python se usa en ciencia de datos, web, y automatizacion.
La comunidad de Python es muy grande y activa.
Aprender Python es una gran decision para tu carrera.
"""

# Tu codigo aqui:


### ✅ Solución Ejercicio 6.3

In [ ]:
# Solucion: Word Count mejorado
import re

texto_ejercicio = """
Python es un lenguaje de programacion muy popular.
Python se usa en ciencia de datos, web, y automatizacion.
La comunidad de Python es muy grande y activa.
Aprender Python es una gran decision para tu carrera.
"""

# Guardamos el texto
ruta = "/home/jovyan/data/sample/texto_python.txt"
with open(ruta, 'w') as f:
    f.write(texto_ejercicio)

def limpiar_palabra(palabra):
    """Elimina caracteres no alfabeticos y convierte a minusculas"""
    # re.sub reemplaza todo lo que NO sea letra con vacio
    return re.sub(r'[^a-záéíóúñ]', '', palabra.lower())

# Word Count mejorado
resultado = sc.textFile(ruta) \
    .flatMap(lambda linea: linea.split()) \
    .map(limpiar_palabra) \
    .filter(lambda p: len(p) >= 3) \
    .map(lambda p: (p, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .filter(lambda x: x[1] > 1) \
    .sortBy(lambda x: x[1], ascending=False)

print("Palabras que aparecen mas de 1 vez (sin palabras cortas):")
for palabra, conteo in resultado.collect():
    print(f"  '{palabra}': {conteo}")

# Output esperado:
# 'python': 4
# 'muy': 2
# ...

---
# === RESUMEN FINAL ===

## Resumen

### Conceptos Clave
- **RDD**: Colección distribuida, resiliente e inmutable de datos
- **Particiones**: Divisiones del RDD distribuidas en workers
- **Transformaciones**: `map`, `filter`, `flatMap`, `reduceByKey` - son lazy
- **Acciones**: `collect`, `count`, `reduce`, `take` - disparan ejecución
- **Pares clave-valor**: Patrón fundamental para agregaciones

### Conexión con AWS
- **Amazon EMR**: Ejecuta jobs de Spark con RDDs en clusters grandes
- **AWS Glue**: Usa DynamicFrames (similar a RDDs) internamente
- **Amazon S3**: Los RDDs pueden leer/escribir directamente a S3

### Siguiente Paso
Continúa con: `02_dataframes_api.ipynb` para aprender la API moderna de DataFrames

In [ ]:
# Limpieza de archivos temporales
import os

archivos = [
    "/home/jovyan/data/sample/ventas_rdd.txt",
    "/home/jovyan/data/sample/texto_wordcount.txt",
    "/home/jovyan/data/sample/texto_python.txt"
]

for archivo in archivos:
    if os.path.exists(archivo):
        os.remove(archivo)

print("Limpieza completada")